# OpenCascade Geometry Processing

OCCT is a collection of 3D CAD libraries written in C++ and released under LGPL. It is "designed for rapid production of sophisticated domain-specific CAD/CAM/CAE applications" and provides the basis for FreeCAD and several open-source BIM projects. Python bindings are provided through at least two projects, [pyOCCT](https://github.com/trelau/pyOCCT) (\~80 github stars, linux/win only) and [pythonocc-core](https://github.com/tpaviot/pythonocc-core) (\~700 github stars, cross-platform including MacOS).

It is a large, complicated, old (begun in 1999) project that isn't widely used and therefore doesn't have extensive informal documentation (tutorials, examples and forums). Using OCCT in Python has the further complication of needing to translate between C++ and Python syntax. While documentation for pythonocc-core is available, it is not hosted online in an accessible format. There is, however, a repository containing [examples](https://github.com/tpaviot/pythonocc-demos) in Python.

The high-level [guide](https://dev.opencascade.org/doc/overview/html/), linked as "full online documentation" on the project website, is extremely helpful for understanding OCCT on a conceptual level, while the [c++ api docs](https://dev.opencascade.org/doc/refman/html/), confusingly linked as "reference manual", provide low-level details on class inheritance, methods and properties. See also the [forums](https://dev.opencascade.org/forums).

Other resources:

- http://opencascade.wikidot.com
- https://opencascade.blogspot.com/2009/02/topology-and-geometry-in-open-cascade_12.html

CGAL, which provides the basis for SFCGAL support in PostGIS and underpins OpenSCAD is a potential alternative, although it is not designed as a full-featured BRep modeling tool and focuses more on meshes. Bindings for CGAL can be generated for other languages through SWIG.

In [1]:
import sys
import re
import math

In [2]:
import shapely.wkt

OCCT is organized into seven modules, imports we'll use here are listed below:

In [3]:
### foundation classes

from OCC.Core.TColgp import TColgp_HArray1OfPnt, TColgp_Array1OfPnt2d, TColgp_Array1OfPnt
from OCC.Core.TColgp import TColgp_SequenceOfPnt
from OCC.Core.GeomAbs import GeomAbs_C2


### modeling data

from OCC.Core.gp import gp_Pnt, gp_Pnt2d, gp_Vec, gp_Dir, gp_Ax2, gp_Pln # primitives
from OCC.Core.Geom2d import Geom2d_Curve
from OCC.Core.Geom import Geom_Curve, Geom_OffsetCurve
from OCC.Core.gce import gce_MakePln, gce_MakeLin # direct construction
from OCC.Core.TopoDS import TopoDS_Iterator
from OCC.Core.BRepAdaptor import BRepAdaptor_CompCurve

### modeling algorithms

# high-level modeling routines (boolean ops, construction of primitives, kinematics)
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge, BRepBuilderAPI_MakeWire, BRepBuilderAPI_MakePolygon, BRepBuilderAPI_MakeFace, BRepBuilderAPI_MakeShell, BRepBuilderAPI_MakeSolid
from OCC.Core.BRepOffsetAPI import BRepOffsetAPI_ThruSections

# low-level mathematical support functions (intersections, projections, interpolations)
from OCC.Core.Geom2dAPI import Geom2dAPI_Interpolate, Geom2dAPI_PointsToBSpline
from OCC.Core.GeomAPI import GeomAPI_PointsToBSpline, GeomAPI_Interpolate, GeomAPI_PointsToBSplineSurface


### visualization


### data exchange


### application framework

from OCC.Core.BOPAlgo import BOPAlgo_MakeConnected, BOPAlgo_BuilderSolid
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Fuse, BRepAlgoAPI_Common
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakePrism
from OCC.Core.ShapeUpgrade import ShapeUpgrade_UnifySameDomain
from OCC.Core.TopTools import TopTools_ListOfShape

### pythonocc-core also gives us some extra stuff

from OCC.Display.WebGl.jupyter_renderer import JupyterRenderer


In [4]:
# add udtools core modules to path for testing
module_path = '../../core/python/'
if module_path not in sys.path:
    sys.path.append(module_path)
    
from geometry.conversion import wkt_to_occ, wire_to_face, curve_to_wire

Input geometry will mostly come from PostGIS in the form of [well-known text](https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry) and potentially later well-known binary. Here's some data for testing:

In [5]:
test_edge = 'LINESTRING(-46.22362966486253 -23.632720552763203,-59.2367360109929 -71.32453354485915)'
test_bounds = 'POLYGON((-46.22362966486253 -23.632720552763203,-59.2367360109929 -71.32453354485915,-160.80903549143113 -43.799661718687275,-154.23228616663255 -19.696363531431416,-147.7965995783452 3.889560617128154,-141.0705573076848 28.540102876344463,-134.68245048471726 51.951466001191875,-33.1094805712346 24.42985526434495,-46.22362966486253 -23.632720552763203))'

Geometry primitives, provided by the [modeling data](https://dev.opencascade.org/doc/overview/html/index.html#intro_overview_moddata) module and prefixed as `gp_*` provide the basis for all geometric objects in OCCT. Boundary representations (BReps) are represented "as an aggregation of geometry within topology" where geometry describes a shape mathematically (curves, surfaces) and topology provides a data structure.

In [6]:
point = (1.0, 1.0, 1.0)
occ_point = gp_Pnt(*point)

The `gce` package provides tools for direct construction of certain primitives, like circles, lines etc.

In [7]:
points = [None, (1.0, 1.0, 1.0), (1.0, 0.0, 0.0)]
gce_MakeLin(gp_Pnt(*points[1]), gp_Pnt(*points[2]))

<class 'gce_MakeLin'>

Compound shapes require OCCT collections as inputs when they're being created. These can be fixed (Array) or variable width (Sequence). Note that in OCCT, arrays are 1-indexed by convention.

In [8]:
points = [None, (1.0, 1.0, 1.0), (1.0, 0.0, 0.0), (2.0, 0.0, 0.0)]

occ_sequence = TColgp_SequenceOfPnt()
for pt in points[1:]:
  occ_pt = gp_Pnt(*pt)
  occ_sequence.Append(occ_pt)

occ_array = TColgp_Array1OfPnt(1, 2)
occ_array.SetValue(1, gp_Pnt(*points[1]))
occ_array.SetValue(2, gp_Pnt(*points[2]))

occ_array_2 = TColgp_Array1OfPnt(1, 4)
occ_array_2.SetValue(1, gp_Pnt(*points[1]))
occ_array_2.SetValue(2, gp_Pnt(*points[2]))
occ_array_2.SetValue(3, gp_Pnt(*points[3]))
occ_array_2.SetValue(4, gp_Pnt(*points[1]))

occ_harray = TColgp_HArray1OfPnt(occ_array) # can make HArray from Array
#occ_harrayfromseq = TColgp_HArray1OfPnt(occ_sequence) # but unclear if/how possible w sequences

These can then be used to create curves (`Geom_BSplineCurve`). See the Wikipedia article on [splines](https://en.wikipedia.org/wiki/Spline_%28mathematics%29) for a detailed explanation of how this is represented mathematically.

In [9]:
# open curve, method 1
interp = GeomAPI_Interpolate(occ_harray, False, 0.01)
interp.Perform()
crv = interp.Curve()

# closed curve
interp_closed = GeomAPI_Interpolate(occ_harray, True, 0.5)
interp_closed.Perform()
crv_closed = interp_closed.Curve()

# open curve, method 2
crv2 = GeomAPI_PointsToBSpline(occ_array).Curve()

# closed curve, method 2
# will be closed if the first and last elements in the array are the same
# will have a higher degree automatically unless a linear spline is specified
# with the degree min/max parameters
crv_closed_2 = GeomAPI_PointsToBSpline(occ_array_2, 1, 1).Curve()

for i, c in enumerate([crv, crv_closed, crv2, crv_closed_2]):
  print(f'Curve {i + 1} is {"closed" if c.IsClosed() else "open"} and has a degree of {c.Degree()}')

Curve 1 is open and has a degree of 1
Curve 2 is closed and has a degree of 1
Curve 3 is open and has a degree of 1
Curve 4 is closed and has a degree of 1


In [10]:
# or surfaces from array2
# srf = GeomAPI_PointsToBSplineSurface(occ_2array, 1, 1, GeomAbs_C2, 0.1)

Edges, wires, polygons faces etc (topology objects) can also be built up from points. Or, they can be converted from geometry objects. Curves can be converted to edges with `BRepBuilderAPI_MakeEdge`, and combined together into a wire with `BrepBuilderAPI_MakeWire`.`BRepAdaptor_CompCurve` can make a curve from a wire?

In [11]:
wire = wkt_to_occ(test_edge)
bounds = wkt_to_occ(test_bounds)
boundsface = wire_to_face(bounds)

In [12]:
# from a curve, you can then build up topological objects like so:
edge = BRepBuilderAPI_MakeEdge(crv_closed_2).Shape()
wire = BRepBuilderAPI_MakeWire(edge).Shape()
face = BRepBuilderAPI_MakeFace(wire).Shape()
face

<class 'TopoDS_Face'>

In [13]:
# adaptors let you use one kind of object as another, here Wire as Curve
adaptor = BRepAdaptor_CompCurve(bounds)
adaptor.Value(0.5)
adaptor.IsClosed()

True

In [24]:
# offset a curve, then make it an edge for display
offset_result = Geom_OffsetCurve(crv2, 1.0, gp_Dir(), False)
offset_edge = BRepBuilderAPI_MakeEdge(offset_result).Shape()

In [33]:
offset_2 = Geom_OffsetCurve(offset_result, 2.0, gp_Dir(0,0,1), False)

In [ ]:
# loft some curves
loft = BRepOffsetAPI_ThruSections(True, True, 0.1)
loft.AddWire(curve_to_wire(crv2))
loft.AddWire(curve_to_wire(offset_result))
loft.AddWire(curve_to_wire(offset_2))
loft_result = loft.Shape()

In [48]:
loft_result.Closed()

True

## Questions

- What is the difference between Array and HArray?
- Can sequences be cast to arrays?
- To confirm, do geometry objects need to be put into a topology structure before they can be rendered? Exported?


In [52]:
# set up renderer
renderer = JupyterRenderer()

In [53]:
renderer.DisplayShape(loft_result, render_edges=True, topo_level="default", shape_color="#abdda4", update=True)